[[<- back to the GitHub Repo](https://github.com/rasbt/pattern_classification)]

In [1]:
%load_ext watermark
%watermark -a 'Gopala KR' -v -p pandas,numpy,scikit-learn -d

Gopala KR 2018-01-28 

CPython 3.6.3
IPython 6.2.1

pandas 0.20.3
numpy 1.13.1
scikit-learn 0.19.0


/srv/venv/lib/python3.6/site-packages/watermark/watermark.py:155: DeprecationWarning: Importing scikit-learn as `scikit-learn` has been depracated and will not be supported anymore in v1.7.0. Please use the package name `sklearn` instead.
  DeprecationWarning)


<br>
<br>

# Tips and Tricks for Encoding Categorical Features in Classification Tasks

Features can come in various different flavors. Typically we distinguish between

- continuous and
- categorical (discrete)
features.


And the categorical features can be categorized further into:

- ordinal and
- nominal (= no order implied) features.



Now, most implementations of machine learning algorithms require numerical data as input, and we have to prepare our data accordingly. This notebook contains some useful tips for how to encode categorical features using Python pandas and scikit-learn.

<br>
<br>

### Sections

- [Example Data](#Example-Data)
- [Class Labels](#Class-Labels)
- [Ordinal Features](#Ordinal-Features)
- [Nominal Features](#Nominal-Features)
- [Inverse Mapping](#Inverse-Mapping)
- [Using scikit-learn and pandas features](#Using-scikit-learn-and-pandas-features)
    - [scikit LabelEncoder](#scikit-LabelEncoder)
    - [scikit DictVectorizer](#scikit-DictVectorizer)
    - [scikit OneHotEncoder](#scikit-OneHotEncoder)
    - [pandas get_dummies](#pandas-get_dummies)

<br>
<br>

## Example Data

[[back to top](#Sections)]

First, let us create a simple example dataset with 3 different kinds of features:
- color: nominal
- size: ordinal
- prize: continuous

In [2]:
import pandas as pd
df = pd.DataFrame([
            ['green', 'M', 10.1, 'class1'], 
            ['red', 'L', 13.5, 'class2'], 
            ['blue', 'XL', 15.3, 'class1']])

df.columns = ['color', 'size', 'prize', 'class label']
df

,color,size,prize,class label
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


<br>
<br>

## Class Labels

[[back to top](#Sections)]

"Typical" machine learning algorithms handle class labels with "no order implied" - unless we use a ranking classifier (e.g., SVM-rank). Thus, it is save to use a simple set-item-enumeration to convert the class labels from a string representation into integers.

In [3]:
class_mapping = {label:idx for idx,label in enumerate(set(df['class label']))}

df['class label'] = df['class label'].map(class_mapping)
df

,color,size,prize,class label
0,green,M,10.1,1
1,red,L,13.5,0
2,blue,XL,15.3,1


<br>
<br>

## Ordinal Features

[[back to top](#Sections)]

Ordinal features need special attention: We have to make sure that the correct values are associated with the corresponding strings. Thus, we need to set-up an explicit mapping dictionary:

In [4]:
size_mapping = {
           'XL': 3,
           'L': 2,
           'M': 1}

df['size'] = df['size'].map(size_mapping)
df

,color,size,prize,class label
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


<br>
<br>

## Nominal Features

[[back to top](#Sections)]

Unfortunately, we can't simply apply the same mapping scheme to the `color` column that we used for the `size`-mapping above. However, we can use another simply trick and convert the "colors" into binary features: Each possible color value becomes a feature column itself (with values 1 or 0).

In [5]:
color_mapping = {
           'green': (0,0,1),
           'red': (0,1,0),
           'blue': (1,0,0)}

df['color'] = df['color'].map(color_mapping)
df

,color,size,prize,class label
0,"(0, 0, 1)",1,10.1,1
1,"(0, 1, 0)",2,13.5,0
2,"(1, 0, 0)",3,15.3,1


In [6]:
import numpy as np
y = df['class label'].values
X = df.iloc[:, :-1].values
X = np.apply_along_axis(func1d= lambda x: np.array(list(x[0]) + list(x[1:])), axis=1, arr=X)

print('Class labels:', y)
print('\nFeatures:\n', X)


Class labels: [1 0 1]

Features:
 [[  0.    0.    1.    1.   10.1]
 [  0.    1.    0.    2.   13.5]
 [  1.    0.    0.    3.   15.3]]


<br>
<br>

## Inverse Mapping

[[back to top](#Sections)]

If we want to convert the features back into its original representation, we can simply do so my using inverted mapping dictionaries:

In [7]:
inv_color_mapping = {v: k for k, v in color_mapping.items()}
inv_size_mapping = {v: k for k, v in size_mapping.items()}
inv_class_mapping = {v: k for k, v in class_mapping.items()}

df['color'] = df['color'].map(inv_color_mapping)
df['size'] = df['size'].map(inv_size_mapping)
df['class label'] = df['class label'].map(inv_class_mapping)
df

,color,size,prize,class label
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


<br>
<br>

## Using scikit-learn and pandas features

[[back to top](#Sections)]

The scikit-learn maching library comes with many useful preprocessing functions that we can use for our convenience.

<br>
<br>

### scikit LabelEncoder

[[back to top](#Sections)]

In [8]:
from sklearn.preprocessing import LabelEncoder

class_le = LabelEncoder()
df['class label'] = class_le.fit_transform(df['class label'])

size_mapping = {
           'XL': 3,
           'L': 2,
           'M': 1}

df['size'] = df['size'].map(size_mapping)
df

,color,size,prize,class label
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


The class labels can be converted back from integer to string via the `inverse_transform` method:

In [9]:
class_le.inverse_transform(df['class label'])

array(['class1', 'class2', 'class1'], dtype=object)

<br>
<br>

### scikit  DictVectorizer

[[back to top](#Sections)]

The [`DictVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html) is another handy tool for feature extraction. The `DictVectorizer` takes a list of dictionary entries (feature-value mappings) and transforms it to vectors. The expected input looks like this:

In [10]:
df.transpose().to_dict().values()

dict_values([{'color': 'green', 'size': 1, 'prize': 10.1, 'class label': 0}, {'color': 'red', 'size': 2, 'prize': 13.5, 'class label': 1}, {'color': 'blue', 'size': 3, 'prize': 15.3, 'class label': 0}])

Note that the dictionary keys in each row represent the feature column labels. 

<br>
<br>

Now, we can use the `DictVectorizer` to turn this
mapping into a matrix:

In [11]:
from sklearn.feature_extraction import DictVectorizer
dvec = DictVectorizer(sparse=False)

X = dvec.fit_transform(df.transpose().to_dict().values())
X

array([[  0. ,   0. ,   1. ,   0. ,  10.1,   1. ],
       [  1. ,   0. ,   0. ,   1. ,  13.5,   2. ],
       [  0. ,   1. ,   0. ,   0. ,  15.3,   3. ]])

<br>
<br>

As we can see in the array above, the columns were reordered during the conversion (due to the hash mapping when we used the dictionary). However, we can simply add back the column names via the `get_feature_names` function.

In [12]:
pd.DataFrame(X, columns=dvec.get_feature_names())

,class label,color=blue,color=green,color=red,prize,size
0,0.0,0.0,1.0,0.0,10.1,1.0
1,1.0,0.0,0.0,1.0,13.5,2.0
2,0.0,1.0,0.0,0.0,15.3,3.0


<br>
<br>

### scikit OneHotEncoder

[[back to top](#Sections)]

Another useful tool in scikit-learn is the [`OneHotEncoder`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html). The idea is the same as in the `DictVectorizer` example above; the only difference is that the `OneHotEncoder` takes integer columns as input. Here we are an `LabelEncoder`, we use the `LabelEncoder` first, to prepare the `color` column before we use the `OneHotEncoder`.

In [13]:
color_le = LabelEncoder()
df['color'] = color_le.fit_transform(df['color'])

df

,color,size,prize,class label
0,1,1,10.1,0
1,2,2,13.5,1
2,0,3,15.3,0


In [14]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)

X = ohe.fit_transform(df[['color']].values)
X

array([[ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.]])

<br>
<br>

### pandas get_dummies

[[back to top](#Sections)]

Also, pandas comes with a convenience function to create new categories for nominal features, namely: [`get_dummies`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.reshape.get_dummies.html).
But first, let us quickly regenerate a fresh example `DataFrame` where the size and class label columns are already taken care of.

In [15]:
import pandas as pd
df = pd.DataFrame([
            ['green', 'M', 10.1, 'class1'], 
            ['red', 'L', 13.5, 'class2'], 
            ['blue', 'XL', 15.3, 'class1']])

df.columns = ['color', 'size', 'prize', 'class label']

size_mapping = {
           'XL': 3,
           'L': 2,
           'M': 1}
df['size'] = df['size'].map(size_mapping)

class_mapping = {label:idx for idx,label in enumerate(set(df['class label']))}
df['class label'] = df['class label'].map(class_mapping)


df

,color,size,prize,class label
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


<br>
<br>

Using the `get_dummies` will create a new column for every unique string in a certain column:

In [16]:
pd.get_dummies(df)

,size,prize,class label,color_blue,color_green,color_red
0,1,10.1,1,0,1,0
1,2,13.5,0,0,0,1
2,3,15.3,1,1,0,0


Note that the `get_dummies` function leaves the numeric columns untouched, how convenient!

In [ ]:
test complete; Gopal